    https://github.com/himoacs/iex_data
    https://iextrading.com/about/press/2017/01/
    https://iextrading.com/developer/
    https://iextrading.com/apps/stocks/#/
    https://iextrading.com/developer/docs/#iex-market-data
    https://github.com/iexg/IEX-API
    
    https://api.tiingo.com/docs/iex/realtime
    http://alanpryorjr.com/2017-09-24-stockstreamer/

    https://github.com/QuantConnect/Lean/releases/tag/v2.3.0.3
    http://blog.intrinio.com/stock-api-developers/
    
    https://github.com/QuantConnect/Lean
    https://github.com/QuantConnect/Lean/tree/master/Algorithm.Python#quantconnect-python-algorithm-project
    https://github.com/QuantConnect/Lean#linux-debian-ubuntu

    https://github.com/yhilpisch/py4fi/tree/master/python36

http://www.enlistq.com/python-api-getting-market-financial-data-iex/

In [38]:
### Not Working need to check iex.data.py file
import sys
sys.path.append('/home/octo/Dropbox/iex_ts')
import iex_data as IEX
iex = IEX.API()
#print(iex.get_latest_quote_and_trade(['AAPL', 'IBM']))
#iex.get_latest_quote_and_trade('AAPL')
#iex.get_latest_trade(['AAPL', 'IBM'])

http://alanpryorjr.com/2017-09-24-stockstreamer/

https://github.com/apryor6/stockstreamer/blob/master/data_fetcher.py

In [22]:
import io
import os
import requests
import pandas as pd
from bokeh.models import ColumnDataSource, HoverTool, ResizeTool, SaveTool, CustomJS
from bokeh.models.widgets import Paragraph, Panel, Tabs, TextInput, Button, DataTable, TableColumn, DateFormatter
from bokeh.models.widgets import TextInput, Button
from bokeh.plotting import figure, curdoc
from bokeh.layouts import row, widgetbox

/home/octo/Downloads/iex-flask/iex.py

In [23]:
TICKER = "SPY"
base = "https://api.iextrading.com/1.0/"
data = ColumnDataSource(dict(time=[], display_time=[], price=[]))

In [24]:
def get_last_price(symbol):
    payload = {
        "format": "csv",
        "symbols": symbol
    }
    endpoint = "tops/last"

    if symbol == "TEST":
        prices_df = get_test_data()
        return prices_df

    raw = requests.get(base + endpoint, params=payload)
    raw = io.BytesIO(raw.content)
    prices_df = pd.read_csv(raw, sep=",")
    prices_df["time"] = pd.to_datetime(prices_df["time"], unit="ms")
    prices_df["display_time"] = prices_df["time"].dt.strftime("%m-%d-%Y %H:%M:%S.%f")

    return prices_df

In [27]:
new_price = get_last_price(symbol=TICKER)

In [34]:
print(get_last_price(symbol=TICKER))

  symbol   price  size                    time                display_time
0    SPY  263.08   200 2018-02-12 15:35:19.006  02-12-2018 15:35:19.006000


https://github.com/addisonlynch/iexfinance/tree/dev/iexfinance

In [44]:
import requests
import json

url = 'https://api.iextrading.com/1.0/deep?symbols=SPY'
r = requests.get(url)
#print(json.loads(r.content))
print(json.loads(r))

TypeError: the JSON object must be str, not 'Response'

In [47]:
import urllib.request
import json
url = 'https://api.iextrading.com/1.0/deep?symbols=SPY'
req = urllib.request.Request(url)

##parsing response
r = urllib.request.urlopen(req).read()
cont = json.loads(r.decode('utf-8'))
counter = 0
print(cont['bids'])

[{'price': 265.24, 'size': 400, 'timestamp': 1518459461132}, {'price': 265.23, 'size': 400, 'timestamp': 1518459459973}, {'price': 265.22, 'size': 400, 'timestamp': 1518459458164}, {'price': 265.21, 'size': 400, 'timestamp': 1518459457876}, {'price': 265.2, 'size': 400, 'timestamp': 1518459456683}, {'price': 265.19, 'size': 400, 'timestamp': 1518459454561}, {'price': 264.94, 'size': 200, 'timestamp': 1518459076688}, {'price': 264.78, 'size': 200, 'timestamp': 1518458903997}, {'price': 264.6, 'size': 100, 'timestamp': 1518454571234}, {'price': 264.55, 'size': 100, 'timestamp': 1518454568709}, {'price': 263.84, 'size': 100, 'timestamp': 1518454510491}, {'price': 263.53, 'size': 100, 'timestamp': 1518455017232}, {'price': 263.1, 'size': 100, 'timestamp': 1518451974943}, {'price': 263.06, 'size': 100, 'timestamp': 1518453348859}, {'price': 262.28, 'size': 100, 'timestamp': 1518459408301}, {'price': 259, 'size': 386, 'timestamp': 1518455692860}, {'price': 258.59, 'size': 100, 'timestamp': 1

In [ ]:

##parcing json
for item in cont['data']['children']:
    counter += 1
    print("Title:", item['data']['title'], "\nComments:", item['data']['num_comments'])
    print("----")

##print formated
#print (json.dumps(cont, indent=4, sort_keys=True))
print("Number of titles: ", counter)

In [1]:
from urllib.request import urlopen
import abc
import time
import datetime
import json
import psycopg2
from threading import Thread
from functools import partial

class StockFetcher(metaclass=abc.ABCMeta):
	"""
	Base class for fetching stock information
	"""
	def __init__(self, stocks):
	    self.stocks = stocks

	@abc.abstractmethod
	def fetchPrice(self, stock):
		"""
		returns current price of stock
		"""
		return NotImplemented

	@abc.abstractmethod
	def fetchStockHighLow(self, stock):
		"""
		returns the high/low values of stock
		"""
		return NotImplemented

	@abc.abstractmethod
	def fetchImageURL(self, stock):
		"""
		returns a URL pointing to the logo corresponding to stock
		"""
		return NotImplemented

class IEXStockFetcher(StockFetcher):
	"""
	Fetches stock information using iextrading.com API
	"""

	url_prefix = "https://api.iextrading.com/1.0/stock/"
	url_suffix_price = "/price"
	url_suffix_img = "/logo"
	url_suffix_highlow = "/quote"

	def __init__(self, stocks):
		super().__init__(stocks)

	def fetchAllPrices(self):
		stock_data = {}
		prices = {}
		stock_data['timestamp'] = datetime.datetime.now()
		threads = []
		for stock in self.stocks:
			t = Thread(target=partial(self.fetchPriceInto, stock, prices))
			threads.append(t)
			t.start()
		for t in threads:
			t.join()
		stock_data['prices'] = prices
		return stock_data

	def fetchAllImages(self):
		urls = {}
		threads = []
		for stock in self.stocks:
			t = Thread(target=partial(self.fetchURLInto, stock, urls))
			threads.append(t)
			t.start()
		for t in threads:
			t.join()
		return urls

	def fetchAllHighLow(self):
		high_low = {}
		threads = []
		for stock in self.stocks:
			t = Thread(target=partial(self.fetchHighLowInto, stock, high_low))
			threads.append(t)
			t.start()
		for t in threads:
			t.join()
		return high_low

	def fetchPriceInto(self, stock, results=None):
		results[stock] = self.fetchPrice(stock)

	def fetchURLInto(self, stock, results=None):
		results[stock] = self.fetchImageURL(stock)

	def fetchHighLowInto(self, stock, results=None):
		results[stock] = self.fetchStockHighLow(stock)

	def fetchPrice(self, stock):
		# get the price of a single stock
		try:
			resp = urlopen("{}{}{}".format(IEXStockFetcher.url_prefix, stock, IEXStockFetcher.url_suffix_price))
			price = float(resp.readlines()[0])
			return price
		except:
			return self.fetchPrice(stock)

	def fetchImageURL(self, stock):
		# get the image url of a single stock
		try:
			resp = urlopen("{}{}{}".format(IEXStockFetcher.url_prefix, stock, IEXStockFetcher.url_suffix_img))
			resp = json.loads(resp.readlines()[0].decode('utf8'))
			return resp['url']
		except:
			return self.fetchImageURL(stock)

	def fetchStockHighLow(self, stock):
		# get the image url of a single stock
		try:
			resp = urlopen("{}{}{}".format(IEXStockFetcher.url_prefix, stock, IEXStockFetcher.url_suffix_highlow))
			resp = json.loads(resp.readlines()[0].decode('utf8'))
			return (resp['week52High'], resp['week52Low'])
		except:
			return self.fetchStockHighLow(stock)

In [2]:
stocks_to_fetch = ['GE', 'AMZN', 'NVDA', 'INTC', 'AAPL', 'NFLX']
stock_fetcher = IEXStockFetcher(stocks_to_fetch)

In [6]:
stock_fetcher.fetchAllPrices()

{'prices': {'AAPL': 165.53,
  'AMZN': 1440.62,
  'GE': 14.765,
  'INTC': 44.535,
  'NFLX': 268.43,
  'NVDA': 239.95},
 'timestamp': datetime.datetime(2018, 2, 14, 23, 16, 1, 394136)}

In [35]:
import pandas as pd
df = pd.DataFrame()

In [41]:
def fetchInsertStockLoop(sleeptime=1):
    """
    main loop for fetching and inserting stocks
    """
    df = pd.DataFrame()
    while True:
        stock_updates =stock_fetcher.fetchAllPrices()
        for stock, price in stock_updates['prices'].items():
            df=df.append(pd.DataFrame({'stock':stock,'price':float(price)},index=[stock_updates['timestamp']]))
        time.sleep(sleeptime)
        print(df)

In [42]:
fetchInsertStockLoop()

                               price stock
2018-02-14 23:43:02.589641   267.560  NFLX
2018-02-14 23:43:02.589641   241.440  NVDA
2018-02-14 23:43:02.589641    14.805    GE
2018-02-14 23:43:02.589641   166.080  AAPL
2018-02-14 23:43:02.589641    44.755  INTC
2018-02-14 23:43:02.589641  1439.560  AMZN
                               price stock
2018-02-14 23:43:02.589641   267.560  NFLX
2018-02-14 23:43:02.589641   241.440  NVDA
2018-02-14 23:43:02.589641    14.805    GE
2018-02-14 23:43:02.589641   166.080  AAPL
2018-02-14 23:43:02.589641    44.755  INTC
2018-02-14 23:43:02.589641  1439.560  AMZN
2018-02-14 23:43:04.633012   267.560  NFLX
2018-02-14 23:43:04.633012   241.440  NVDA
2018-02-14 23:43:04.633012    14.805    GE
2018-02-14 23:43:04.633012   166.080  AAPL
2018-02-14 23:43:04.633012    44.755  INTC
2018-02-14 23:43:04.633012  1439.560  AMZN
                               price stock
2018-02-14 23:43:02.589641   267.560  NFLX
2018-02-14 23:43:02.589641   241.440  NVDA
2018-02-14 

KeyboardInterrupt: 

In [38]:
df

""


https://github.com/addisonlynch/iexfinance/tree/dev/iexfinance